#### Creating the DataFrames to visualize the data

In [1]:
import os
import pandas as pd

In [4]:
root_dir = r'D:\Nuvem\OneDrive - Indra\Área de Trabalho\DATAGLOWUP_34\downloads'

# List all the file names to be searched in each subfolder
file_names = ['listings.csv', 'listings_2.csv', 'calendar_2.csv', 'neighbourhoods.csv', 'reviews.csv', 'reviews_2.csv']

# Dictionary to store the combined DataFrames for each file name
combined_dataframes = {}

# Iterate over the file names
for filename in file_names:
    # List to store the DataFrames from each subfolder for a specific file
    dataframes = []
    # Iterate over the subfolders
    for foldername, _, filenames in os.walk(root_dir):
        # Check if the file is present in the subfolder
        if filename in filenames:
            # Construct the full path to the file
            filepath = os.path.join(foldername, filename)
            # Read the file and store it in a DataFrame
            df = pd.read_csv(filepath, low_memory=False)
            # Remove columns with all null values
            df = df.dropna(axis=1, how='all')
            # Add a new column with the city name (subfolder name)
            df['City'] = os.path.basename(foldername)
            # Add the DataFrame to the list
            dataframes.append(df)
    # Combine all DataFrames into a single DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)
    # Store the combined DataFrame in the dictionary
    combined_dataframes[filename] = combined_df

In [4]:
# Show the combined DataFrames for each file
for filename, df in combined_dataframes.items():
    print(f'\nCombined DataFrame for {filename}:')
    print(df.info())
    print('\nNulls per column:')
    print(df.isnull().sum())


DataFrame combinado para listings.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194436 entries, 0 to 194435
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              194436 non-null  int64  
 1   name                            194436 non-null  object 
 2   host_id                         194436 non-null  int64  
 3   host_name                       194436 non-null  object 
 4   neighbourhood                   194436 non-null  object 
 5   latitude                        194436 non-null  float64
 6   longitude                       194436 non-null  float64
 7   room_type                       194436 non-null  object 
 8   price                           183729 non-null  float64
 9   minimum_nights                  194436 non-null  int64  
 10  number_of_reviews               194436 non-null  int64  
 11  last_review                     144754

In [7]:
# Show the first lines for each combined DataFrame
for filename, df in combined_dataframes.items():
    print(f'\nCombined DataFrame for {filename}:')
    print(df.head())


DataFrame combinado para listings.csv:
       id                                               name  host_id  \
0   15526  Villa in Solto Collina · ★4.83 · 2 bedrooms · ...    60754   
1   15542  Villa in Solto Collina · ★4.91 · 1 bedroom · 2...    60754   
2  116423  Rental unit in Bossico · ★4.73 · 2 bedrooms · ...   588254   
3  179345  Rental unit in Bossico · ★4.75 · 1 bedroom · 4...   588254   
4  215933  Loft in Bergamo · ★4.90 · 1 bedroom · 2 beds ·...   816380   

  host_name  neighbourhood   latitude  longitude        room_type  price  \
0    Andrea  Solto Collina  45.783200  10.026600  Entire home/apt  135.0   
1    Andrea  Solto Collina  45.782840  10.024570  Entire home/apt   99.0   
2    Angela        Bossico  45.828400  10.044760  Entire home/apt   67.0   
3    Angela        Bossico  45.827220  10.046020  Entire home/apt   77.0   
4  Marcella        Bergamo  45.698385   9.675597     Private room  100.0   

   minimum_nights  number_of_reviews last_review  reviews_per_mo

#### Creating the Data Base

In [ ]:
# Installing the necessary libraries
!pip install sqlalchemy psycopg2-binary pandas

In [5]:
import sqlite3
import psycopg2
from sqlalchemy import create_engine

In [ ]:
# Database name
database_name = 'Airbnd_Italia.db'

# Mapping of pandas data types to SQLite data types
type_mapping = {
    'int64': 'INTEGER',
    'float64': 'REAL',
    'object': 'TEXT',
    'bool': 'INTEGER',
    'datetime64[ns]': 'TEXT',
    'timedelta[ns]': 'TEXT'
}

# Function to create a table automatically from a DataFrame
def create_table_from_dataframe(df, table_name, conn):
    cursor = conn.cursor()
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        sql_type = type_mapping.get(str(dtype), 'TEXT')
        columns.append(f"{col} {sql_type}")
    columns_sql = ", ".join(columns)
    # Drop the table if it already exists to recreate it
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    create_table_query = f"CREATE TABLE {table_name} ({columns_sql})"
    cursor.execute(create_table_query)

# Function to insert data from a DataFrame into the corresponding table
def insert_data_from_dataframe(df, table_name, conn, chunk_size=1000):
    cursor = conn.cursor()
    columns = ", ".join(df.columns)
    placeholders = ", ".join(["?" for _ in df.columns])
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    for start in range(0, len(df), chunk_size):
        end = start + chunk_size
        chunk = df.iloc[start:end]
        data_tuples = [tuple(x) for x in chunk.to_numpy()]
        cursor.executemany(insert_query, data_tuples)
    conn.commit()

# Connect to the database
conn = sqlite3.connect(database_name)

# Create tables and insert data for all DataFrames
for filename, df in combined_dataframes.items():
    table_name = filename.replace('.csv', '')
    create_table_from_dataframe(df, table_name, conn)
    insert_data_from_dataframe(df, table_name, conn)

# Close the connection
conn.close()